In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

DATASET_PATH = '../data-new/data-without-troponina.csv'
df = pd.read_csv(DATASET_PATH)

In [1]:
# Asumptions
NUMERICAL_FEATURES = ['EDAD','PESO Kg', 'ALTURA cm', 'CREAT', 'GLUCEMIA INGR', 'GB', 'TAS INGRESO', 'FC INGRESO']
NUMERICAL_CONTINOUS = NUMERICAL_FEATURES

CATEGORICAL_MULTI_CLASS_FEATURES = ['KILLIP Ingreso', 'FUNCION VENTRICULAR IZQ', 'DIAGNOSTICO', 'NUMERO VASOS', 'Peor KILLIP']
CATEGORICAL_BINARY_FEATURES = ['NUMERO VASOS', 'SEX0', 'HTA', 'DBT', 'DLP', 'TABAQ', 'ANTEC IAM / Angina inestable', 'ANTEC BY PASS', 'ANTEC ATC', 'ANTEC ACE', 'ANTEC INS RENAL', 'ACV TIA PREV', 'EPOC', 'CLAUD ITTE', 'BB PREV', ' IECA/AT2  PREV', 'B CA PREV', 'AAS PREV', 'HIPOGLUCEMIANTES', 'DIURETICOS', 'INSULINA', 'ECG INFRA ST', 'ECG SUPRA ST', 'ECG INV T', 'BRI', 'BRD', 'ECG MCP', 'ECG FA', 'tropst', 'TnT Ultrasensible', 'Elevacion troponina T', 'AAS', 'BB', 'IECA', 'AT2', 'Clopi - prasu - tica', 'HEP sc o iv', 'IIb IIIa', 'B CA', 'ESTATINAS', 'HIPOGLUC METFORM', 'HIPOGL SULFAN', 'HIPOGL GLITAZ', 'INSULINA', 'INS + HIPOGL', 'INOTROPICOS', 'PRUEBA FUNC.', 'PRUEBA FUNC DE ALTO RIESGO', 'CCG', 'TRONCO', 'ATC PRIMARIA', 'ATC INTRAHOSP', 'TROMBOL', 'CRM', 'IAM HOSP(SI INTERNO POR ANGINA) O REIAM', 'APIAM', 'ANGINA REFRACT o RECURR', ' ACV/TIA', 'SANGRADO MAYOR*', 'I RENAL AGUDA']

CATEGORY_FEATURES = CATEGORICAL_BINARY_FEATURES + CATEGORICAL_MULTI_CLASS_FEATURES

FEATURES = NUMERICAL_CONTINOUS + CATEGORY_FEATURES

TARGET = "MUERTE HOSP"

In [3]:
len(FEATURES)

73

Vamos a hacer una selección de features basado en la importancia de las features.

Para ello, vamos a implementar Información Mutua (MI). La información mutua mide la dependencia no lineal entre las variables independientes (features) y la variable objetivo. Se basa en la teoría de la información y cuantifica cuánto conocimiento sobre la variable objetivo se obtiene al conocer una característica específica.


Primero hacemos una imputacion de los valores faltantes provisoria para poder calcular la información mutua.

In [3]:
from sklearn.impute import SimpleImputer

df_copy = df.copy()

median_imputer = SimpleImputer(strategy='median')
mode_imputer = SimpleImputer(strategy='most_frequent')

median_imputer.fit(df_copy[NUMERICAL_FEATURES])
mode_imputer.fit(df_copy[CATEGORY_FEATURES])

df_copy[NUMERICAL_FEATURES] = median_imputer.transform(df_copy[NUMERICAL_FEATURES])
df_copy[CATEGORY_FEATURES] = mode_imputer.transform(df_copy[CATEGORY_FEATURES])


In [4]:
# Separar variables independientes (X) y variable objetivo (y)
X = df_copy[FEATURES]  # Variables predictoras
y = df_copy[TARGET]  # Variable objetivo

In [5]:
from sklearn.feature_selection import mutual_info_classif

# Calcular Información Mutua
mi_scores = mutual_info_classif(X, y, random_state=42)
mi_scores = pd.Series(mi_scores, index=X.columns)

# Mostrar la importancia de cada variable
print("Importancia de Información Mutua:")
print(mi_scores.sort_values(ascending=False))

Importancia de Información Mutua:
INOTROPICOS                0.033373
KILLIP Ingreso             0.030335
I RENAL AGUDA              0.023975
Peor KILLIP                0.020337
FUNCION VENTRICULAR IZQ    0.018713
                             ...   
HEP sc o iv                0.000000
B CA                       0.000000
ESTATINAS                  0.000000
HIPOGLUC METFORM           0.000000
tropst                     0.000000
Length: 73, dtype: float64


In [6]:
# Seleccionar variables con Información Mutua mayor a 0.01
selected_mi_mask = mi_scores.apply(lambda x: x > 0.01)
selected_mi = mi_scores[selected_mi_mask]
print(selected_mi.sort_values(ascending=False))


INOTROPICOS                0.033373
KILLIP Ingreso             0.030335
I RENAL AGUDA              0.023975
Peor KILLIP                0.020337
FUNCION VENTRICULAR IZQ    0.018713
TAS INGRESO                0.017224
GLUCEMIA INGR              0.016233
CREAT                      0.016123
GB                         0.014049
DIURETICOS                 0.012903
CRM                        0.012531
NUMERO VASOS               0.012410
SANGRADO MAYOR*            0.011180
ANGINA REFRACT o RECURR    0.010444
dtype: float64


In [7]:
from sklearn.ensemble import RandomForestClassifier

# Entrenar un modelo Random Forest para evaluar importancia de características
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X, y)

# Obtener importancia de variables
rf_importances = pd.Series(model_rf.feature_importances_, index=X.columns)

# Seleccionar las variables más importantes
selected_rf_mask = rf_importances.apply(lambda x: x > 0.01)
selected_rf = rf_importances[selected_rf_mask]
print("Variables seleccionadas por Random Forest:\n", selected_rf.sort_values(ascending=False))


Variables seleccionadas por Random Forest:
 INOTROPICOS                                0.069633
TAS INGRESO                                0.062337
I RENAL AGUDA                              0.060307
GB                                         0.058136
CREAT                                      0.057678
GLUCEMIA INGR                              0.052957
KILLIP Ingreso                             0.043440
FC INGRESO                                 0.042813
Peor KILLIP                                0.038906
EDAD                                       0.038835
PESO Kg                                    0.036603
ALTURA cm                                  0.031199
BB                                         0.027895
SANGRADO MAYOR*                            0.019945
FUNCION VENTRICULAR IZQ                    0.019536
ECG SUPRA ST                               0.017681
CRM                                        0.017514
IAM HOSP(SI INTERNO POR ANGINA) O REIAM    0.016886
NUMERO VASOS        

In [8]:
# Crear sets de características seleccionadas por cada método
features_mi = set(selected_mi.index)
features_rf = set(selected_rf.index)

# Comparar intersección (variables seleccionadas por varios métodos)
intersect_features = features_mi & features_rf
print("Variables seleccionadas por TODOS los métodos:\n", list(intersect_features))


Variables seleccionadas por TODOS los métodos:
 ['SANGRADO MAYOR*', 'I RENAL AGUDA', 'GB', 'FUNCION VENTRICULAR IZQ', 'TAS INGRESO', 'Peor KILLIP', 'CREAT', 'INOTROPICOS', 'CRM', 'GLUCEMIA INGR', 'KILLIP Ingreso', 'NUMERO VASOS']


Finalmente, vamos a seleccionar las variables que están en la intersección de los dos métodos.


In [9]:
selected_features = list(intersect_features)
print("Variables seleccionadas:\n", selected_features)

df_selected = df[selected_features + [TARGET]]

Variables seleccionadas:
 ['SANGRADO MAYOR*', 'I RENAL AGUDA', 'GB', 'FUNCION VENTRICULAR IZQ', 'TAS INGRESO', 'Peor KILLIP', 'CREAT', 'INOTROPICOS', 'CRM', 'GLUCEMIA INGR', 'KILLIP Ingreso', 'NUMERO VASOS']


In [10]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1818 entries, 0 to 1817
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   SANGRADO MAYOR*          1814 non-null   float64
 1   I RENAL AGUDA            1816 non-null   float64
 2   GB                       1752 non-null   float64
 3   FUNCION VENTRICULAR IZQ  1810 non-null   float64
 4   TAS INGRESO              1806 non-null   float64
 5   Peor KILLIP              1818 non-null   int64  
 6   CREAT                    1808 non-null   float64
 7   INOTROPICOS              1800 non-null   float64
 8   CRM                      1815 non-null   float64
 9   GLUCEMIA INGR            1735 non-null   float64
 10  KILLIP Ingreso           1818 non-null   int64  
 11  NUMERO VASOS             1816 non-null   float64
 12  MUERTE HOSP              1818 non-null   int64  
dtypes: float64(10), int64(3)
memory usage: 184.8 KB


In [11]:
from utils import print_column_with_nan
print_column_with_nan(df_selected)

SANGRADO MAYOR* NaNs: 4 - 0.22002200220022003%
I RENAL AGUDA NaNs: 2 - 0.11001100110011001%
GB NaNs: 66 - 3.6303630363036303%
FUNCION VENTRICULAR IZQ NaNs: 8 - 0.44004400440044006%
TAS INGRESO NaNs: 12 - 0.6600660066006601%
Peor KILLIP NaNs: 0 - 0.0%
CREAT NaNs: 10 - 0.5500550055005501%
INOTROPICOS NaNs: 18 - 0.9900990099009901%
CRM NaNs: 3 - 0.16501650165016502%
GLUCEMIA INGR NaNs: 83 - 4.565456545654565%
KILLIP Ingreso NaNs: 0 - 0.0%
NUMERO VASOS NaNs: 2 - 0.11001100110011001%
MUERTE HOSP NaNs: 0 - 0.0%


In [13]:
df_selected.to_csv('../data-new/data-selected-features.csv', index=False)